In [8]:
import warnings
import requests
from datetime import datetime, timedelta
import json
import time
from geopy.geocoders import Nominatim
from pprint import pprint
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pickle as pk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib
from sklearn.metrics import (
            accuracy_score,
            precision_score, 
            recall_score,
            f1_score,
            roc_auc_score,
            confusion_matrix,
            precision_recall_curve,
            auc,
            mean_absolute_error,
            mean_squared_error,
            r2_score)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

warnings.filterwarnings("ignore", message="This Pipeline instance is not fitted yet")

class GLDASFetcher:
    def __init__(self, username=None, password=None):
        """
        Initialize GLDAS/POWER fetcher (username/password only needed for GLDAS OPeNDAP, 
        not for NASA POWER API).
        """
        self.username = username
        self.password = password

        # GLDAS variable names mapped to NASA POWER parameters
        self.variables_map = {
            'temp': ['T2M_MAX', 'T2M_MIN'],           # Daily max/min temp
            'humidity': ['QV2M'],                     # Specific humidity
            'pressure': ['PS'],                       # Surface pressure
            'precipitation': ['PRECTOTCORR'],         # Corrected daily precipitation
            'solar_rad': ['ALLSKY_SFC_SW_DWN'],       # Solar radiation
            'wind_speed': ['WS2M']                    # Wind speed
        }

    def get_data(self, lat, lon, start_date, end_date, variables=None):
        """
        Fetch daily NASA POWER data for a given location & date range.
        """
        variables = variables or list(self.variables_map.keys())
    
        base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    
        # Collect POWER parameter codes
        power_params = []
        for var in variables:
            power_params.extend(self.variables_map.get(var, []))
    
        params = {
            'parameters': ','.join(power_params),
            'community': 'RE',
            'longitude': lon,
            'latitude': lat,
            'start': start_date.replace("-", ""),
            'end': end_date.replace("-", ""),
            'format': 'JSON'
        }
    
        print(f"🌍 Fetching NASA POWER data for ({lat}, {lon}) from {start_date} to {end_date} ...")
        response = requests.get(base_url, params=params, timeout=60)
    
        if response.status_code != 200:
            print(f"❌ API error {response.status_code}")
            return pd.DataFrame()
    
        data = response.json()['properties']['parameter']
    
        # Build dataframe manually
        records = {}
        for var, timeseries in data.items():
            for date_str, value in timeseries.items():
                if date_str not in records:
                    records[date_str] = {}
                records[date_str][var] = value
    
        # Convert dict -> DataFrame
        df = pd.DataFrame.from_dict(records, orient='index')
        df.index = pd.to_datetime(df.index, format="%Y%m%d")  # Proper date parsing
        df.index.name = "date"
        df.reset_index(inplace=True)
        
        # Add metadata
        df['latitude'] = lat
        df['longitude'] = lon

        columns = ["date","temp_max","temp_min","humidity_specific","pressure","precipitation_total","solar_radiation","wind_speed","lat","lon"]
        df.columns = columns
        print(f"✅ Retrieved {len(df)} daily records")
        return df

    def get_bulk_data(self, locations, start_date, end_date, variables=None):
        """
        Fetch data for multiple locations.

        Args:
            locations (list): [(lat, lon), ...]
            start_date (str): YYYY-MM-DD
            end_date (str): YYYY-MM-DD
            variables (list): Variables list

        Returns:
            dict: { "lat_lon": DataFrame }
        """
        results = {}
        for lat, lon in locations:
            df = self.get_data(lat, lon, start_date, end_date, variables)
            if not df.empty:
                key = f"lat_{lat}_lon_{lon}"
                results[key] = df
        return results

    def to_csv(self, data, filename):
        """Export single DataFrame or dict of DataFrames to CSV"""
        if isinstance(data, dict):
            all_data = []
            for loc, df in data.items():
                df_copy = df.copy()
                df_copy['location'] = loc
                all_data.append(df_copy)
            combined = pd.concat(all_data, ignore_index=True)
            combined.to_csv(filename, index=False)
        else:
            data.to_csv(filename, index=False)




    def get_location_by_address(self, address):
        """Return location data from an address, retrying if failed."""
        time.sleep(1)
        geolocator = Nominatim(user_agent="gldas_fetcher")
        try:
            return geolocator.geocode(address).raw
        except:
            return self.get_location_by_address(address)  # Recursive retry


    def main():
        """Example usage with NASA data"""
        
        print("🚀 NASA GLDAS Data Fetcher")
        print("=" * 50)
        
        
        username = "mahmoudmo12"
        password = "Mahmoudmetawe12@"
        
    
        # Initialize with real credentials
        fetcher = GLDASFetcher(username=username, password=password)
        city = input("enter the city: ")
        
        
        location = fetcher.get_location_by_address(city)
        lat = location["lat"]
        lon = location["lon"]
        city_name = location['display_name']
        #start_date = input("enter the start date formula (yyyy-mm-dd): ")
        #end_date = input("enter the end date formula (yyyy-mm-dd): ")
        
        # Test with a single location
        print(f"\n🌍 Fetching DAILY data for {city_name}")
        
        data = fetcher.get_data(
            lat=lat,
            lon=lon,
            start_date="1984-01-01",
            end_date="2025-10-20",
            variables=['temp', 'humidity', 'pressure', 'precipitation', 'solar_rad', 'wind_speed']
        )
        
        if not data.empty:
            print(f"\n✅ SUCCESS! Retrieved {len(data)} real data points")
            print(f"📋 Columns: {list(data.columns)}")
            
            # Export real data
            fetcher.to_csv(data, "nasa_daily_weather_data.csv")
            # import the data
            df = pd.read_csv("nasa_daily_weather_data.csv")
        else:
            print("❌")     
        return df 



class WeatherForecaster:
    def __init__(self, data_path, pipeline_path="pipeline_ensemble.pkl"):
        self.data_path = data_path
        self.pipeline_path = pipeline_path
        self.df, _ = self._wrangle(data_path, 'humidity_specific')
        self.columns = self.df.columns
        self.targets = self.columns.drop(["date", "day_of_year", "lat", "lon", "did_rain"])
        self.models = None
        self.pipeline = None
        self.best_threshold = None

    #wrangle without noise columns
    def _wrangle(self,path,target=None):
        
        df = pd.read_csv(path)
        df['date'] = pd.to_datetime(df['date'])
        day_of_year = df['date'].dt.dayofyear
        df['day_of_year'] = day_of_year
        # getting the target column with the date for the model 
        df_prophet = df[['date', target]].rename(columns={'date': 'ds', target: 'y'})

        did_rain = [0 if i < 0.2 else 1 for i in df['precipitation_total']]
        df['did_rain'] = did_rain   
        
         # the nan val in nasa api is -999
        df.replace(-999.0000,np.nan,inplace=True)
        df_prophet.replace(-999.0000,np.nan,inplace=True)
        #drop noise
        df.drop(columns=["wind_speed","precipitation_total"],inplace=True)
        # drop nulls
        
        df.dropna(inplace=True)
        df_prophet.dropna(inplace=True)
        
        return df,df_prophet

    def train_and_save_prophet(self):
        for target in self.targets:
            _, df_prophet = self._wrangle(self.data_path, target)
            model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            model.fit(df_prophet)
            filename = f'prophet_model_{target}.pkl'
            with open(filename, 'wb') as file:
                pk.dump(model, file)
            print(f"model for {target} is saved")

    def gathering_models(self):
        all_models = []
        for target in self.targets:
            filename = f'prophet_model_{target}.pkl'
            with open(filename, 'rb') as file:
                all_models.append(pk.load(file))
        self.models = all_models
        return all_models

    def predict_func(self, start_date, interval):
        
        """
        Internal helper to forecast all weather variables for a given time interval.

        Args:
            start_date (str): The starting date in 'YYYY-MM-DD' format.
            interval (int): The number of future days to forecast.

        Returns:
            pd.DataFrame: A DataFrame containing the forecasted weather variables,
                          ready to be used by the rain prediction pipeline.
        """
        # 1. Create a DataFrame with the future dates for Prophet
        future_dates_df = pd.DataFrame({
            'ds': pd.date_range(start=start_date, periods=interval, freq='D')
        })

        # 2. Start building the final DataFrame, beginning with the date column
        future_weather_df = pd.DataFrame({'date': future_dates_df['ds']})
        # 3. Loop through each Prophet model to forecast its specific weather variable
        #    This assumes self.targets and self.models are in the same order.
        print("Forecasting future weather conditions...")
        for target_variable, model in zip(self.targets, self.models):
            # Use the model to predict values for the entire date range
            forecast = model.predict(future_dates_df)

            # Extract the forecasted values ('yhat') and the dates ('ds')
            future_values = forecast[['ds', 'yhat']].rename(
                columns={'ds': 'date', 'yhat': target_variable}
            )

            # Merge this forecast into our main weather DataFrame
            future_weather_df = pd.merge(future_weather_df, future_values, on='date')
        
        # 4. Ensure the column order matches exactly what the pipeline was trained on
        required_columns = ['date', 'temp_max', 'temp_min', 'humidity_specific', 'pressure', 'solar_radiation']
        future_weather_df = future_weather_df[required_columns]

        print("Weather forecast complete.")
        return future_weather_df

    class ProphetWrapper(BaseEstimator, TransformerMixin):
        def __init__(self, model):
            self.model = model

        def fit(self, X, y=None):
            return self

        def transform(self, X):
            future = pd.DataFrame({'ds': X.flatten()})
            forecast = self.model.predict(future)
            return forecast[['yhat']].values

    def save_pipeline(self):
        
        # 1. Define your new feature set (after creating lags, etc.)
        # The date is still needed for Prophet, other features for XGBoost
        date_feature = ['date'] 
        weather_features = ['temp_max', 'temp_min', 'humidity_specific', 'pressure', "solar_radiation"] 
        
        # 2. Create the Prophet forecasting part (same as your code)
        p_models = self.models
        wrapped_prophets = [
            (f'prophet_{i}', self.ProphetWrapper(model))
            for i, model in enumerate(p_models)]
        prophet_forecasters = FeatureUnion(wrapped_prophets)
        prophet_pipeline = Pipeline([
            ('selector', ColumnTransformer([('date_selector', 'passthrough', [0])])), # Select only the date column
            ('prophet_features', prophet_forecasters)
        ])
        # Features and target
        X = self.df.drop(columns='did_rain')
        y = self.df['did_rain'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        
        neg, pos = np.bincount(y)   # count 0s and 1s in did_rain
        imbalance_ratio = neg / pos
        # 3. Create a pipeline for your standard numerical weather features
        weather_pipeline = Pipeline([
            ('selector', ColumnTransformer([('weather_selector', 'passthrough', [i for i, col in enumerate(X.columns) if col in weather_features])])),
            ('scaler', StandardScaler()) # Scaling is good practice for XGBoost
        ])
        
        # 4. Combine them using FeatureUnion
        combined_features = FeatureUnion([
            ('prophet_pipeline', prophet_pipeline),
            ('weather_pipeline', weather_pipeline)
        ])
        
        # 5. Create the final, complete pipeline
        final_pipeline = Pipeline([
            ('features', combined_features),
            ('xgb_classifier', XGBClassifier(n_estimators=550,
                                             max_depth=8,
                                             learning_rate=0.05,
                                             subsample=0.7,
                                             scale_pos_weight=imbalance_ratio,
                                             random_state=42)) # Your XGBoost model here
        ])
        #over_sampler = RandomOverSampler(sampling_strategy="all",random_state=42)
        #X_train_over,y_train_over = over_sampler.fit_resample(X_train,y_train)
        # Now fit this final_pipeline on your data that includes ALL columns
        final_pipeline.fit(X_train, y_train)
        
        joblib.dump(final_pipeline, self.pipeline_path)

        # Evaluation 
        y_pred = final_pipeline.predict(X_test)
        y_proba = final_pipeline.predict_proba(X_test)[:, 1]
        
        
        precision, recall, _ = precision_recall_curve(y_test, y_proba)
        pr_auc = auc(recall, precision)
        # After fitting the model and getting y_proba
        precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
        
        # Calculate F1 score for each threshold
        # Add a small epsilon to avoid division by zero :1e-9
        f1_scores = 2 * recall * precision / (recall + precision + 1e-9)
        
        # Find the threshold that gives the best F1 score
        best_threshold = thresholds[np.argmax(f1_scores)]
        
        print(f"Best Threshold for F1 Score: {best_threshold*100} %")
        
        self.best_threshold = best_threshold
        
        y_pred = (y_proba >= best_threshold).astype(int)
        
        print("\n📊 Model Evaluation on Test Set")
        

        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Precision:", precision_score(y_test, y_pred))
        print("Recall:", recall_score(y_test, y_pred))
        print("F1 Score:", f1_score(y_test, y_pred))
        print("ROC AUC:", roc_auc_score(y_test, y_proba))
        print("PR AUC:", pr_auc)
        
        print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred)) 
    def calculate_metrics(self,test, train, model):     
        # predictions
        train_pred = model.predict(train)
        test_pred = model.predict(test)
    
        y_true = test['y'].values
        y_pred = test_pred['yhat'].values
    
        y_train = train['y'].values
        yhat_train = train_pred['yhat'].values
    
        smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))
    
        metrics = {
            'MAE': mean_absolute_error(y_true, y_pred),
            'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            'R2': r2_score(y_true, y_pred),
            "SMAPE": smape,
            'Mean_Error': np.mean(y_pred - y_true),  # Bias
            'Training MAE': mean_absolute_error(y_train, yhat_train)
        }
        
        for metric, value in metrics.items():
            print(f"{metric}: {value:.2f}")

        return metrics
    def model(self):
        """
        Main function to generate a rain and weather forecast for a specified interval.
        """
        # 1. Load the main rain prediction pipeline
        self.pipeline = joblib.load(self.pipeline_path)
        
        # 2. Get the date and interval from the user
        start_date = input("Enter the start date (YYYY-MM-DD): ")
        interval = int(input("Enter the number of days to forecast: "))
        print("=" * 50)

        # 3. Call the helper function to get the weather forecast for the full interval.
        #    This `weather_preds_df` contains all the features needed for the next step.
        weather_preds_df = self.predict_func(start_date, interval)

        # 4. Use the complete weather forecast to predict the probability of rain.
        #    The pipeline receives a DataFrame with multiple rows and 6 columns, just as it expects.
        rain_probabilities = self.pipeline.predict_proba(weather_preds_df)

        # 5. Format the rain probability results into a clean DataFrame
        proba_df = pd.DataFrame(
            (rain_probabilities * 100).round(2),
            columns=["Prob. of No Rain (%)", "Prob. of Rain (%)"],
            index=weather_preds_df['date']  # Use the future dates as the index
        )
        
        best_threshold_percent = self.best_threshold * 100
        proba_df["Recommendation"] = proba_df['Prob. of Rain (%)'].apply(
            lambda x: "Take an umbrella! ☔" if x >= best_threshold_percent else "Enjoy the clear skies! ☀️")
        

        # 7. Return both the rain probabilities and the detailed weather predictions
        return proba_df, weather_preds_df.set_index('date')
    
    def Prophet_metrics(self):
        # metrics calculation for each model of the prophets models
        test_list = []
        train_list = []
        
        for i in self.targets:
            _,df_prophet = self._wrangle("nasa_daily_weather_data.csv",i)
        
            # Split the data for the prophet models metrics (temp,humidity,...)
            train_df, test_df = train_test_split(df_prophet, test_size=0.2, shuffle=False)
        
            train_list.append(train_df)
        
            test_list.append(test_df)
        models = self.gathering_models()
        for i in range(len(self.targets)):
            print("="*70)
            print(f"Evaluation Results {self.targets[i]}")
            wf.calculate_metrics(test_list[i],train_list[i],models[i])
            print("="*70)



In [9]:
#fetch the data according to the location 
GLDASFetcher.main()

wf = WeatherForecaster("nasa_daily_weather_data.csv")

# Train Prophet models and save them
wf.train_and_save_prophet()

# Load Prophet models into memory
wf.gathering_models()

# Save ensemble pipeline
wf.save_pipeline()

wf.Prophet_metrics()
# Make predictions
proba_df, weather_preds = wf.model()

print("\nWeather predictions:")
print(weather_preds)
print("\nRain probabilities:")
print(proba_df)



🚀 NASA GLDAS Data Fetcher


enter the city:  سرس الليان



🌍 Fetching DAILY data for سرس الليان, المنوفية, 32861, مصر
🌍 Fetching NASA POWER data for (30.4439168, 30.9707264) from 1984-01-01 to 2025-10-20 ...
✅ Retrieved 15269 daily records

✅ SUCCESS! Retrieved 15269 real data points
📋 Columns: ['date', 'temp_max', 'temp_min', 'humidity_specific', 'pressure', 'precipitation_total', 'solar_radiation', 'wind_speed', 'lat', 'lon']


20:24:11 - cmdstanpy - INFO - Chain [1] start processing
20:24:16 - cmdstanpy - INFO - Chain [1] done processing


model for temp_max is saved


...

model for pressure is saved


20:24:46 - cmdstanpy - INFO - Chain [1] start processing
20:24:49 - cmdstanpy - INFO - Chain [1] done processing


model for solar_radiation is saved
Best Threshold for F1 Score: 42.76325607299805 %

📊 Model Evaluation on Test Set
Accuracy: 0.9050425671250819
Precision: 0.5849056603773585
Recall: 0.5406976744186046
F1 Score: 0.5619335347432024
ROC AUC: 0.8451434180039475
PR AUC: 0.5649016525912377
Confusion Matrix:
 [[2578  132]
 [ 158  186]]
Evaluation Results temp_max
MAE: 2.15
RMSE: 2.87
R2: 0.87
SMAPE: 7.52
Mean_Error: 0.01
Training MAE: 2.25
Evaluation Results temp_min
MAE: 1.41
RMSE: 1.82
R2: 0.90
SMAPE: 10.63
Mean_Error: 0.01
Training MAE: 1.41
Evaluation Results humidity_specific
MAE: 0.91
RMSE: 1.15
R2: 0.78
SMAPE: 11.29
Mean_Error: -0.01
Training MAE: 0.87
Evaluation Results pressure
MAE: 0.22
RMSE: 0.30
R2: 0.64
SMAPE: 0.22
Mean_Error: 0.01
Training MAE: 0.24
Evaluation Results solar_radiation
MAE: 0.38
RMSE: 0.59
R2: 0.90
SMAPE: 7.73
Mean_Error: 0.00
Training MAE: 0.42


Enter the start date (YYYY-MM-DD):  2025-11-01
Enter the number of days to forecast:  7


Forecasting future weather conditions...
Weather forecast complete.

Weather predictions:
             temp_max   temp_min  humidity_specific    pressure  \
date                                                              
2025-11-01  30.193481  17.228217           9.538518  101.215459   
2025-11-02  30.014445  17.080371           9.483051  101.222270   
2025-11-03  29.837041  16.931954           9.427627  101.228998   
2025-11-04  29.660950  16.783262           9.372107  101.235632   
2025-11-05  29.485819  16.634568           9.316350  101.242168   
2025-11-06  29.311272  16.486122           9.260215  101.248603   
2025-11-07  29.136922  16.338146           9.203565  101.254938   

            solar_radiation  
date                         
2025-11-01         4.355070  
2025-11-02         4.316422  
2025-11-03         4.278837  
2025-11-04         4.242278  
2025-11-05         4.206702  
2025-11-06         4.172049  
2025-11-07         4.138254  

Rain probabilities:
            Pro

In [10]:
weather_preds

,temp_max,temp_min,humidity_specific,pressure,solar_radiation
date,,,,,
2025-11-01,30.193481,17.228217,9.538518,101.215459,4.355070
2025-11-02,30.014445,17.080371,9.483051,101.222270,4.316422
2025-11-03,29.837041,16.931954,9.427627,101.228998,4.278837
2025-11-04,29.660950,16.783262,9.372107,101.235632,4.242278
2025-11-05,29.485819,16.634568,9.316350,101.242168,4.206702
2025-11-06,29.311272,16.486122,9.260215,101.248603,4.172049
2025-11-07,29.136922,16.338146,9.203565,101.254938,4.138254


In [11]:
proba_df

,Prob. of No Rain (%),Prob. of Rain (%),Recommendation
date,,,
2025-11-01,99.589996,0.41,Enjoy the clear skies! ☀️
2025-11-02,99.260002,0.74,Enjoy the clear skies! ☀️
2025-11-03,94.800003,5.20,Enjoy the clear skies! ☀️
2025-11-04,95.570000,4.43,Enjoy the clear skies! ☀️
2025-11-05,95.980003,4.02,Enjoy the clear skies! ☀️
2025-11-06,97.889999,2.11,Enjoy the clear skies! ☀️
2025-11-07,93.730003,6.27,Enjoy the clear skies! ☀️
